In [11]:
import pandas as pd
import usaddress
import pyodbc
from datamart import fix_me

In [12]:
username = 'vigrose'
password_edw = 'Gryffindor~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [27]:
ADDRESSES = \
    f'''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE,
    P.POST_CD AS ZIP,
    P.SRC_POST_KEY AS comm_id
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.CAT_CD CC
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL
    AND
    CC.CAT_CD_ID = A.CAT_CD_ID;
    '''
addresses = pd.read_sql(con=AMAEDW, sql=ADDRESSES)
addresses.head()

ENTITY_ID = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = '9'
    """
entity_ids = pd.read_sql(con=AMAEDW, sql=ENTITY_ID)
entity_ids.head()

#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)
party_ids.head()

NPI_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS NPI
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 38
    AND
    P.ACTIVE_IND = 'Y'
    """
npi = pd.read_sql(con=AMAEDW, sql=NPI_QUERY)
npi.head()

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,COMM_ID
0,1807827,9128232,10666 N Torrey Pines Rd,,,La Jolla,CA,92037,715505
1,1808279,9128232,10666 N Torrey Pines Rd,,,La Jolla,CA,92037,715505
2,1809712,9128232,10666 N Torrey Pines Rd,,,La Jolla,CA,92037,715505
3,1810451,9128232,10666 N Torrey Pines Rd,,,La Jolla,CA,92037,715505
4,1813185,9128232,10666 N Torrey Pines Rd,,,La Jolla,CA,92037,715505


In [2]:
other_bolos = pd.read_csv('../../Data/PhoneAppend/POLO_top_address_mismatch_2021-02-16.csv')

In [3]:
other_bolos.head()

,entity_id,polo_comm_id,polo_address_key,polo_score,top_comm_id,address_key,score,addr_count,diff
0,1234632,22635755,4429 CLARA ST STE 400_70115,0.527964,780722,1514 JEFFERSON HWY_70121,0.611437,5,0.083473
1,1234645,19012700,300 N AIRPORT RD STE 2_35504,0.628181,8382257,800 MONTCLAIR RD_35213,0.697305,4,0.069124
2,1234658,15249990,208 MCFARLAND CIR N STE 100_35406,0.340417,14908481,208 MCFARLAND CIR N_35406,0.836914,5,0.496498
3,1234671,15536864,2880 DAUPHIN ST_36606,0.688162,6648848,2525 TELEPHONE RD_39567,0.846508,4,0.158346
4,1234710,14909987,8000 AL HIGHWAY 69_35976,0.528468,14421347,36 MIMOSA ST NE_35016,0.625044,2,0.096576


In [4]:
bolos = pd.read_excel('../../Data/PhoneAppend/mismatch_phone_match_diff_address.xlsx')

In [6]:
bolos.columns = [c.replace(' ','_') for c in bolos.columns.values]

In [14]:
good_bolos = other_bolos[other_bolos.entity_id.isin(bolos.entity_id)==False]

In [29]:
address_comm = addresses.drop_duplicates(subset = 'COMM_ID')[['ADDR_1','ADDR_2','ADDR_3','CITY','STATE','ZIP','COMM_ID']]

In [36]:
good_bolos.entity_id

0          1234632
1         14851105
2          2016478
3          2383286
4         43988954
            ...   
300798    14621070
300799    14621083
300800    15424145
300801    14621174
300802    14621200
Name: entity_id, Length: 300803, dtype: int64

In [41]:
good_bolos['ENTITY_ID']=[str(x) for x in good_bolos.entity_id]

In [32]:
good_bolos = pd.merge(good_bolos, address_comm, left_on='top_comm_id', right_on='COMM_ID').drop_duplicates()

In [43]:
good_bolos = pd.merge(good_bolos, entity_ids, on='ENTITY_ID')

In [20]:
for row in bolos.itertuples():
    if row.PHONE!=row.Phone_Number:
        print('whoops')

In [250]:
gov = pd.read_csv('../../Data/DataGov/DAC_NationalDownloadableFile_090521.csv', encoding='latin')

In [251]:
gov['NPI']=[str(x) for x in gov.NPI]

In [252]:
gov = pd.merge(gov, npi, on='NPI')

In [52]:
all_file = pd.merge(gov, good_bolos, on='PARTY_ID')

In [254]:
gov = pd.merge(gov, party_ids, on='PARTY_ID')

In [257]:
gov.to_csv('../../Data/DataGov/me_gov.csv', index=False)

In [86]:
def clean_address_one(add_1):
    if add_1 == 'None':
        addr_1 = ''
    elif add_1 == 'NAN':
        addr_1 = ''
    else:
        addr_1 = add_1 + ','
    return(addr_1)

def is_a_match(thing_1, thing_2):
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    else:
        match = False
    return(match)

def error_handle(parsed_string):
    new_dict = {}
    for thing in parsed_string:
        if thing[1] in new_dict.keys():
            a_list = [new_dict[thing[1]], thing[0]]
            new_dict[thing[1]] = max(a_list, key=len)
        else:
            new_dict[thing[1]] = thing[0]
    return(new_dict)

def get_all_keys(moar):
    all_keys =[]
    for row in moar.itertuples():
        bolo_addr_1 = clean_address_one(row.ADDR_2)
        bolo_address = f'{bolo_addr_1}{row.ADDR_1} {row.CITY} {row.STATE}'
        try:
            bolo_dict = usaddress.tag(bolo_address)[0]
        except usaddress.RepeatedLabelError as e:
            print(e.original_string)
            bolo_dict = error_handle(e.parsed_string)
            print('')
        nat_addr_1 = clean_address_one(row.adr_ln_2)
        nat_address = f'{nat_addr_1}{row.adr_ln_1}, {row.cty}, {row.st}'
        try:
            nat_dict = usaddress.tag(nat_address)[0]
        except usaddress.RepeatedLabelError as e:
            print(e.original_string)
            nat_dict = error_handle(e.parsed_string)
            print('')
        le_keys = list(nat_dict.keys()) + list(bolo_dict.keys())
        for key in le_keys:
            if key not in all_keys:
                all_keys.append(key)
        return (all_keys)
    
def parse_address(moar, all_keys):
    dict_list = []
    for row in moar.itertuples():
        new_dict = {}
        new_dict['PARTY_ID'] = row.PARTY_ID
        new_dict['adrs_id'] = row.adrs_id
        bolo_addr_1 = clean_address_one(row.ADDR_2)
        bolo_address = f'{bolo_addr_1}{row.ADDR_1}, {row.CITY}, {row.STATE}'
        try:
            bolo_dict = usaddress.tag(bolo_address)[0]
        except usaddress.RepeatedLabelError as e:
            print(e.original_string)
            bolo_dict = error_handle(e.parsed_string)
            print('')
        nat_addr_1 = clean_address_one(row.adr_ln_2)
        nat_address = f'{nat_addr_1}{row.adr_ln_1}, {row.cty}, {row.st}'.replace('  ','')
        try:
            nat_dict = usaddress.tag(nat_address)[0]
        except usaddress.RepeatedLabelError as e:
            print(e.original_string)
            nat_dict = error_handle(e.parsed_string)
            print('')
        for key in all_keys:
            new_dict[key] = 'None'
            if key in bolo_dict.keys() and key in nat_dict.keys():
                new_dict[key] = is_a_match(nat_dict[key], bolo_dict[key])
        dict_list.append(new_dict)
    return (dict_list)

In [56]:
[c.replace(' ','_') for c in all_file.columns.values]

In [84]:
all_file = all_file.rename(columns={'_adrs_id':'adrs_id','_adr_ln_2':'adr_ln_2','_cty':'cty','_adr_ln_1':'adr_ln_1','_st':'st'})
all_file.columns

Index(['NPI', '_Ind_PAC_ID', '_Ind_enrl_ID', '_lst_nm', '_frst_nm', '_mid_nm',
       '_suff', '_gndr', '_Cred', '_Med_sch', '_Grd_yr', '_pri_spec',
       '_sec_spec_1', '_sec_spec_2', '_sec_spec_3', '_sec_spec_4',
       '_sec_spec_all', '_org_nm', '_org_pac_id', '_num_org_mem', 'adr_ln_1',
       'adr_ln_2', '_ln_2_sprs', 'cty', 'st', '_zip', '_phn_numbr',
       '_hosp_afl_1', '_hosp_afl_lbn_1', '_hosp_afl_2', '_hosp_afl_lbn_2',
       '_hosp_afl_3', '_hosp_afl_lbn_3', '_hosp_afl_4', '_hosp_afl_lbn_4',
       '_hosp_afl_5', '_hosp_afl_lbn_5', '_ind_assgn', '_grp_assgn', 'adrs_id',
       'PARTY_ID', 'entity_id', 'polo_comm_id', 'polo_address_key',
       'polo_score', 'top_comm_id', 'address_key', 'score', 'addr_count',
       'diff', 'ADDR_1', 'ADDR_2', 'ADDR_3', 'CITY', 'STATE', 'ZIP', 'COMM_ID',
       'ENTITY_ID'],
      dtype='object')

In [76]:
LE_KEYS = get_all_keys(all_file)
new_list = parse_address(all_file, LE_KEYS)

In [79]:
LE_KEYS

['OccupancyType',
 'OccupancyIdentifier',
 'AddressNumber',
 'StreetName',
 'StreetNamePostType',
 'PlaceName',
 'StateName']

In [78]:
all_file[['ADDR_1','ADDR_2']]

,ADDR_1,ADDR_2
0,401 Bogle St,Ste 207
1,2680 W Avalon Dr,
2,446 Hungry Harbor Rd,
3,446 Hungry Harbor Rd,
4,446 Hungry Harbor Rd,
...,...,...
443061,1 Cavalcade Ct,
443062,49 Urb Lakeview Est,
443063,11109 Leo Collins Dr,None
443064,11109 Leo Collins Dr,None


In [80]:
all_file = all_file.fillna('None')

In [87]:
new_list = parse_address(all_file, LE_KEYS)

250 APEX MEDICAL SERVICES,7625 WISCONSIN AVE, BETHESDA, MD

RAINBOW GARDENS ALF 2,1801 NE 182ND ST, NORTH MIAMI BEACH, FL

3170 AVE MILITAR CARR 2 BO MORA, ISABELA, PR

Pl# 50,2 Bala Plz, Bala Cynwyd                 , PA

2 BALA PLAZA SUITE PL50,333 E CITY LINE AVE, BALA CYNWYD, PA

Pl# 50,2 Bala Plz, Bala Cynwyd                 , PA

Pl# 50,2 Bala Plz, Bala Cynwyd                 , PA

EINSTEIN CT SURGERY ASSOC,5501 OLD YORK RD WCB 4TH FLR, PHILADELPHIA, PA

Corner 177 Street,386 Fort Washington Ave, New York                    , NY

4911 W ST JOE HWY 102, LANSING, MI

100 JOHN W PFLUG MD PC,2323 S 171ST ST, OMAHA, NE

SUITE 403 4TH FLOOR,1 SEARS DR, PARAMUS, NJ

212 SEE RIGHT NOW,653 N TOWN CTR DR, LAS VEGAS, NV

354th Mdg,2630 Central Ave, Eielson AFB, AK

354th Mdg,2630 Central Ave, Eielson AFB, AK

202 INSTITUTE OF UROLOGY,2160 W 86TH ST, INDIANAPOLIS, IN

25A SUITE 207,3130 N COUNTY RD, TROY, OH

203 CLEAR LAKE MEDICAL CLINIC,450 W MEDICAL CTR BLVD, WEBSTER, TX

SUITE 125,4646 N 

2nd Fl,92 Bay 11th St, Brooklyn                    , NY

2nd Fl,92 Bay 11th St, Brooklyn                    , NY

Galloway Rd, Ste A130,7800 SW 87th Ave, Miami                       , FL

Galloway Rd, Ste A130,7800 SW 87th Ave, Miami                       , FL

Galloway Rd, Ste A130,7800 SW 87th Ave, Miami                       , FL

Galloway Rd, Ste A130,7800 SW 87th Ave, Miami                       , FL

Galloway Rd, Ste A130,7800 SW 87th Ave, Miami                       , FL

Galloway Rd, Ste A130,7800 SW 87th Ave, Miami                       , FL

51 SUITE 1200,1816 S FM, DECATUR, TX

SUITE 350,1710 E SAUNDERS ST B, LAREDO, TX

Ste 800,4365 N Expressway # 7783, Brownsville                 , TX

83 SUITE 800,4365 S, BROWNSVILLE, TX

SUITE 104 BLUE BLDG,2000 WASHINGTON ST, NEWTON, MA

105 AMBULATORY UROSURGICAL CENTER,401 E JEFFERSON ST, ROCKVILLE, MD

2nd Floor Ste 2401,531 Washington St, Watertown                   , NY

212 LASER LIGHT TREATMENT CENTER,165 ROWLAND WAY, NOVATO, CA


Ste 120,4090 Mapleshade Ln Ste 220, Plano                       , TX

104 B RIVERVIEW PLAZA,33 S DELAWARE AVE, YARDLEY, PA

497 Mt. Prospect Avenue 181 Franklin Ave,Ste 201, Nutley                      , NJ

497 Mt. Prospect Avenue 181 Franklin Ave,Ste 201, Nutley                      , NJ

497 Mt. Prospect Avenue 181 Franklin Ave,Ste 201, Nutley                      , NJ

497 Mt. Prospect Avenue 181 Franklin Ave,Ste 201, Nutley                      , NJ

497 Mt. Prospect Avenue 181 Franklin Ave,Ste 201, Nutley                      , NJ

SUITE 108,80 1 CARR 2 KM, ARECIBO, PR

M14 SUITE B 1,1007 W AVE, PALMDALE, CA

3935 I 49 S SERVICE RD, OPELOUSAS, LA

BLDG 2 UNIT A,1103 W SHERMAN AVE, VINELAND, NJ

544 SUITE 401,211 E FM, MURPHY, TX

4THAVENUE SUITE 205,9920, BROOKLYN, NY

SUITE 64,HC 33 BOX, LAS VEGAS, NM

820 SUITE C,7904 NE LOOP, NORTH RICHLAND HILLS, TX

St. Louis Associates In Ob/Gyn,621 S New Ballas Rd Ste 584A, Saint Louis, MO

200 CARR 2 SUITE 206,TORRE MEDICA DR PEDRO BLANCO

21 SUITE 212,1789 CARR, SAN JUAN, PR

Est De San Fernando,D1 Calle 6, Carolina                    , PR

Road 506 Km 1 Torre San Cristobal Suite 312, Coto Laurel, PR

1 TORRE SAN CRISTOBAL OFICINA 312,ROAD 506 KM, COTO LAUREL, PR

400 SOUTHEAST TEXAS OPHTHALMOLOGY,1323 S 27TH ST, NEDERLAND, TX

PO Box 9397,9166 Svl Box, Victorville                 , CA

205A,1995 CARR 2 METRO MEDICAL CENTE, BAYAMON, PR

SUITE 100,1800 AL HWY 157, CULLMAN, AL

79 Middleville Rd 11768,35 Paddock Dr, Northport                   , NY

# 312,46-001 Kam Hwy Ste 312, Kaneohe                     , HI

131 OLD RD TO 9 ACRE COR, CONCORD, MA

5 HOSPITAL AUXILIO MUTUO,AVE JUAN PONCE DE LEON PARADA, HATO REY, PR

3009 SUITE 120,6032 FM, SCHERTZ, TX

76 134,10833 LE CONTE AVE, LOS ANGELES, CA

105 434,1141 N LOOP 1604, SAN ANTONIO, TX

T43 THE KENMORE,5415 CONNECTICUT NW AVE, WASHINGTON, DC

1183 OPHTHALMOLOGY DEPARTMENT OF MOUNT,5 E 98TH ST 7TH FL BOX, NEW YORK, NY

METRO MEDICAL NORTH,2415 N KINGSHIGHWAY BLVD, ST L

202 SUBN GASTRO ASSOC,1010 W CHESTER PKE, HAVERTOWN, PA

Dept Psy Rm 3-128 D-30,301 University Blvd, Galveston                   , TX

Dept Psy Rm 3-128 D-30,301 University Blvd, Galveston                   , TX

54 SUITE 201,10820 STATE RD, TRINITY, FL

WOMENS PLACE EAST Ste 214,4214 Medical Center Dr, Fayetteville                , NY

WOMENS PLACE EAST Ste 214,4214 Medical Center Dr, Fayetteville                , NY

104 INTERVENTIONAL CLINIC,7900 W JEFFERSON BLVD, FORT WAYNE, IN

106 ANIMAS URGENT CARE,450 S CAMINO DEL RIO, DURANGO, CO

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKWY, CHICAGO, IL

177 MURDOCK,1653 W CONGRESS PKW

...

Suite 104,15770 Paul Vega MD Dr Ste 104, Hammond, LA

Suite 104,15770 Paul Vega MD Dr Ste 104, Hammond, LA

Suite 104,15770 Paul Vega MD Dr Ste 104, Hammond, LA

1626 BLDG 1A,211 B N FM, BUDA, TX

RR1 Box 11,PO Box 233, Cairo                       , IL

405 TAMPA BAY WOMENS CARE,4321 N MACDILL AVE, TAMPA, FL

104 TAMPA BAY WOMENS CARE,10909 W LINEBAUGH AVE, TAMPA, FL

205 TAMPA BAY WOMENS CARE,4321 N MACDILL AVE, TAMPA, FL

54 SUITE 406,9332 STATE RD, TRINITY, FL

200 TAMPA BAY WOMENS CARE,4150 N ARMENIA AVE, TAMPA, FL

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PINES RD, LA JOLLA, CA

2ND FLOOR SW209,10666 N TORREY PIN

,2103 Caminito Circulo Norte, La Jolla                    , CA

East Paris Medical Bldg,1000 E Paris Ave SE Ste 260, Grand Rapids                , MI

350 AHMG CARDIOLOGY VASCULAR AND CARDIOT,9119 W 74TH ST, MERRIAM, KS

390 AHMG PULMONOLOGY AT SHAWNEE MISSION,8901 W 74TH ST, MERRIAM, KS

350 AHMG CARDIOLOGY VASCULAR AND CARDIOT,9119 W 74TH ST, MERRIAM, KS

110 AHMG GENERAL SURGERY AT SHAWNEE MISSION,9301 W 74TH ST, MERRIAM, KS

130 AHMG GYNECOLOGIC ONCOLOGY AT SHAWNE,9301 W 74TH ST, MERRIAM, KS

208 ADNTHLTH OB GYN,2301 S CLEAR CRK RD, KILLEEN, TX

204 ADVENTHLTH ORTH POD,2301 S CLEAR CRK RD, KILLEEN, TX

Jericho Road Community Health Center,1021 Broadway St, Buffalo                     , NY

Ste 101,4665 Douglas Cir NW # 100, Canton                      , OH

Fmlh East,9200 W Wisconsin Ave, Milwaukee                   , WI

Ste 2531,94 Old Short Hills Rd Ste 1, Livingston                  , NJ

Ste 2531,94 Old Short Hills Rd Ste 1, Livingston                  , NJ

400 E 71st Street 

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

Medical Arts Pavilion 2 Ste 3301,4735 Ogletown Stanton Rd Ste 3301, Newark                      , DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

6 CCHS INFECTIOUS DISEASE,501 W 14TH ST FL, WILMINGTON, DE

4154 CCHS CHIEFS SURGICAL SERVICES,501 W 14TH ST WILMINGTO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

St Louis Childrens Hosp,1 Childrens Pl # 3S35, Saint Louis                 , MO

8115 DEPT OF OTOLARYNGOLOGY,660 S EUCLID AVE BOX, SAINT LOUIS, MO

St Louis Childrens Hosp,1 Childrens Pl # 3S35, Saint Louis                 , MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

Cb 8116 - Nwt 8,1 Childrens Pl, Saint Louis                 , MO

Ste 8B,4921 Parkview Pl Ste 8B, Saint Louis, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTER

8115 DEPT OF OTOLARYNGOLOGY,660 S EUCLID AVE BOX, SAINT LOUIS, MO

8064 DEPARTMENT OF OBSTETRICS AND GYNECOLOGY,660 S EUCLID AVE CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8015 DEPARTMENT OF INTERNAL MEDICINE,660 S EUCLID CAMPUS BOX, SAINT LOUIS, MO

8134 DEPARTMENT OF PSYCHIATRY,660 S EUCLID AVE CAMPU

In [88]:
results = pd.DataFrame(new_list)

In [89]:
with_results = pd.merge(results, all_file, on = ['PARTY_ID', 'adrs_id'])

In [104]:
with_results[['ADDR_1','adr_ln_1','PARTY_ID','_phn_numbr']]

,ADDR_1,adr_ln_1,PARTY_ID,_phn_numbr
0,401 Bogle St,401 BOGLE ST,2715286,6.0668e+09
1,2680 W Avalon Dr,4036 CTR RD,3451635,None
2,446 Hungry Harbor Rd,210 E SUNRISE HWY,2807366,None
3,446 Hungry Harbor Rd,900 FRANKLIN AVE,2807366,None
4,446 Hungry Harbor Rd,900 FRANKLIN AVE,2807366,None
...,...,...,...,...
462511,1 Cavalcade Ct,515 IRON BRIDGE RD,3271075,7.32457e+09
462512,49 Urb Lakeview Est,BF4 CALLE LA PINTA,2573791,7.87539e+09
462513,11109 Leo Collins Dr,2000 TRANS MOUNTAIN RD,2052502,9.15578e+09
462514,11109 Leo Collins Dr,1801 N OREGON ST,2052502,9.15521e+09


In [113]:
with_results['SHORT_ZIP']=[str(x)[0:5] for x in with_results['_zip']]

In [121]:
zip_matches = []
for row in with_results.itertuples():
    ZIP_MATCH = is_a_match(row.SHORT_ZIP,row.ZIP)
    zip_matches.append(ZIP_MATCH)

In [122]:
with_results['ZipCode']=zip_matches

In [126]:
thing_1 = with_results[(with_results.StreetName==True)&(with_results.AddressNumber==True)&(with_results['_phn_numbr']!='None')].drop_duplicates('PARTY_ID')

In [127]:
thing_2 = with_results[(with_results.StreetName==False)&(with_results.AddressNumber==True)&(with_results.ZipCode==True)&(with_results['_phn_numbr']!='None')].drop_duplicates('PARTY_ID')

In [74]:
for row in all_file.head().itertuples():
        bolo_addr_1 = clean_address_one(row.PHYSICAL_ADDR_2)
        bolo_address = f'{bolo_addr_1}{row.PHYSICAL_ADDR_1}, {row.PHYSICAL_CITY}, {row.PHYSICAL_STATE}'
        print(bolo_address)
        try:
            bolo_dict = usaddress.tag(bolo_address)[0]
        except usaddress.RepeatedLabelError as e:
            print(e.original_string)
            bolo_dict = error_handle(e.parsed_string)
            print('')
        print(bolo_dict)

STE 2,300 N AIRPORT RD, JASPER, AL
OrderedDict([('OccupancyType', 'STE'), ('OccupancyIdentifier', '2'), ('AddressNumber', '300'), ('StreetNamePreDirectional', 'N'), ('StreetName', 'AIRPORT'), ('StreetNamePostType', 'RD'), ('PlaceName', 'JASPER'), ('StateName', 'AL')])
2880 DAUPHIN ST, MOBILE, AL
OrderedDict([('AddressNumber', '2880'), ('StreetName', 'DAUPHIN'), ('StreetNamePostType', 'ST'), ('PlaceName', 'MOBILE'), ('StateName', 'AL')])
2880 DAUPHIN ST, MOBILE, AL
OrderedDict([('AddressNumber', '2880'), ('StreetName', 'DAUPHIN'), ('StreetNamePostType', 'ST'), ('PlaceName', 'MOBILE'), ('StateName', 'AL')])
2880 DAUPHIN ST, MOBILE, AL
OrderedDict([('AddressNumber', '2880'), ('StreetName', 'DAUPHIN'), ('StreetNamePostType', 'ST'), ('PlaceName', 'MOBILE'), ('StateName', 'AL')])
2880 DAUPHIN ST, MOBILE, AL
OrderedDict([('AddressNumber', '2880'), ('StreetName', 'DAUPHIN'), ('StreetNamePostType', 'ST'), ('PlaceName', 'MOBILE'), ('StateName', 'AL')])


In [77]:
some_matches.columns

Index(['ME', 'adrs_id', 'OccupancyType', 'OccupancyIdentifier',
       'AddressNumber', 'StreetNamePreDirectional', 'StreetName',
       'StreetNamePostType', 'PlaceName', 'StateName', 'NPI', 'Ind_PAC_ID',
       'Ind_enrl_ID', 'lst_nm', 'frst_nm', 'mid_nm', 'suff', 'gndr', 'Cred',
       'Med_sch', 'Grd_yr', 'pri_spec', 'sec_spec_1', 'sec_spec_2',
       'sec_spec_3', 'sec_spec_4', 'sec_spec_all', 'org_nm', 'org_pac_id',
       'num_org_mem', 'adr_ln_1', 'adr_ln_2', 'ln_2_sprs', 'cty', 'st', 'zip',
       'phn_numbr', 'hosp_afl_1', 'hosp_afl_lbn_1', 'hosp_afl_2',
       'hosp_afl_lbn_2', 'hosp_afl_3', 'hosp_afl_lbn_3', 'hosp_afl_4',
       'hosp_afl_lbn_4', 'hosp_afl_5', 'hosp_afl_lbn_5', 'ind_assgn',
       'grp_assgn', 'entity_id', 'polo_comm_id', 'polo_address_key',
       'polo_score', 'top_comm_id', 'address_key', 'score', 'addr_count',
       'diff', 'Phone_Number', 'Address', 'City', 'State', 'Address1',
       'dhc_match_polo', 'dhc_match_top', 'ME10', 'PHYSICAL_ADDR_1',
     

In [78]:
some_matches['data_phone']=[str(x).replace('.0','') for x in some_matches.phn_numbr]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
offs = []
for row in some_matches.itertuples():
    off_match = False
    if row.data_phone == str(row.PHONE):
        off_match = True
    offs.append(off_match)

In [81]:
some_matches['PHONE_MATCH'] = offs

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
some_matches[some_matches.PHONE_MATCH==True]

,ME,adrs_id,OccupancyType,OccupancyIdentifier,AddressNumber,StreetNamePreDirectional,StreetName,StreetNamePostType,PlaceName,StateName,...,dhc_match_polo,dhc_match_top,ME10,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHONE,data_phone,PHONE_MATCH
4,102000081,AL366062457MO2880XSTXX300,None,None,True,None,True,False,None,None,...,True,False,10200008,2880 DAUPHIN ST,NAN,MOBILE,AL,2514731900,2514731900,True
26,102000269,MS392164634JA970XXDRXX305,False,True,True,None,True,True,True,True,...,True,False,10200026,970 LAKELAND DR,STE 61,JACKSON,MS,6019827850,6019827850,True
29,102000307,FL325012316PE1000XSTXX400,None,None,True,True,True,True,True,True,...,False,True,10200030,1000 W MORENO ST,NAN,PENSACOLA,FL,8504344011,8504344011,True
30,102000366,AL355018952JA3400XEXXX403,False,True,True,None,True,None,True,True,...,True,False,10200036,3400 HIGHWAY 78 E,STE 412,JASPER,AL,2053843013,2053843013,True
35,102000790,AL350964723LI1100XRDXX400,None,None,True,None,True,True,True,True,...,True,False,10200079,1100 PATTON CHAPEL RD,NAN,LINCOLN,AL,2057631414,2057631414,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91482,95708890082,IL606113167CH355XXSTXX400,None,None,True,True,True,True,True,True,...,False,True,9570889008,355 E ERIE ST,NAN,CHICAGO,IL,3122381000,3122381000,True
91489,95709900014,NY103044117ST7XXXXSTXX300,None,None,True,None,True,True,True,True,...,False,True,9570990001,7 REMSEN ST,NAN,STATEN ISLAND,NY,7188878566,7188878566,True
91498,95712950014,MN558052107DU915XXSTXX400,None,None,True,True,True,True,True,True,...,True,False,9571295001,915 E 1ST ST,NAN,DULUTH,MN,2182495555,2182495555,True
91501,96501040018,WI535322909DE4200XDRXX300,None,None,True,None,True,True,False,True,...,True,False,9650104001,4200 SAVANNAH DR,NAN,DEFOREST,WI,6084173300,6084173300,True


In [84]:
19734/len(some_matches)

0.5914226631102586

In [86]:
other_matches = with_results[(with_results.StreetName==True)&(with_results.AddressNumber==True)&(with_results.dhc_match_top==True)]
other_matches['data_phone']=[str(x).replace('.0','') for x in other_matches.phn_numbr]
offs = []
for row in other_matches.itertuples():
    off_match = False
    if row.data_phone == str(row.PHONE):
        off_match = True
    offs.append(off_match)
other_matches['PHONE_MATCH'] = offs

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [90]:
len(other_matches[other_matches.PHONE_MATCH==True])/10674

0.6445568671538318

In [88]:
len(other_matches)

10674

In [129]:
all_things = pd.concat([thing_1, thing_2])

In [135]:
all_things.sort_values('diff', ascending=False)

,ADDR_1,adr_ln_1
461667,305 S Platte Clay Way,305 S PLATTE CLAY WAY
240725,3344 N Futrall Dr,3344 N FUTRALL DR
205934,200 Harry S Truman Pkwy,200 HARRY S TRUMAN PKWY
18131,137 Miracle Dr,137 MIRACLE DR
334001,927 Broadway St Ste 130,927 BROADWAY ST
...,...,...
57069,825 Eastlake Ave E,825 EASTLAKE AVE E
277705,34052 La Plaza,34052 LA PLZ
436963,289 W Huntington Dr,289 W HUNTINGTON DR
271354,1633 Church St,1633 CHURCH ST


In [136]:
all_things.columns

Index(['PARTY_ID', 'adrs_id', 'OccupancyType', 'OccupancyIdentifier',
       'AddressNumber', 'StreetName', 'StreetNamePostType', 'PlaceName',
       'StateName', 'NPI', '_Ind_PAC_ID', '_Ind_enrl_ID', '_lst_nm',
       '_frst_nm', '_mid_nm', '_suff', '_gndr', '_Cred', '_Med_sch', '_Grd_yr',
       '_pri_spec', '_sec_spec_1', '_sec_spec_2', '_sec_spec_3', '_sec_spec_4',
       '_sec_spec_all', '_org_nm', '_org_pac_id', '_num_org_mem', 'adr_ln_1',
       'adr_ln_2', '_ln_2_sprs', 'cty', 'st', '_zip', '_phn_numbr',
       '_hosp_afl_1', '_hosp_afl_lbn_1', '_hosp_afl_2', '_hosp_afl_lbn_2',
       '_hosp_afl_3', '_hosp_afl_lbn_3', '_hosp_afl_4', '_hosp_afl_lbn_4',
       '_hosp_afl_5', '_hosp_afl_lbn_5', '_ind_assgn', '_grp_assgn',
       'entity_id', 'polo_comm_id', 'polo_address_key', 'polo_score',
       'top_comm_id', 'address_key', 'score', 'addr_count', 'diff', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE', 'ZIP', 'COMM_ID', 'ENTITY_ID',
       'SHORT_ZIP', 'ZipCode'],
      dt

In [188]:
data_bolo_sample = all_things[['PARTY_ID', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE', 'ZIP', 'COMM_ID', 'ENTITY_ID','_phn_numbr']].head(5000)

In [148]:
all_things.to_csv('../../Data/PhoneAppend/DataGovMatches.csv', index=False)

In [147]:
good_bolos[good_bolos.ENTITY_ID.isin(all_things.ENTITY_ID)==False].sort_values('diff', ascending=False).head(6000).to_csv('../../Data/PhoneAppend/Bing_Sample.csv', index=False)

,entity_id,polo_comm_id,polo_address_key,polo_score,top_comm_id,address_key,score,addr_count,diff,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,COMM_ID,ENTITY_ID,PARTY_ID
51537,2686628,25337396,NaN,0.098469,6670736,1901 SE 18TH AVE STE 101_34471,0.908892,5,0.810424,1901 SE 18th Ave Ste 101,,,Ocala,FL,34471,6670736,2686628,1879653
292206,14119478,15506320,275 MARTINE ST STE 301_02723,0.089303,2044129,15 OAK MANOR DR_02806,0.887461,6,0.798158,15 Oak Manor Dr,,,Barrington,RI,02806,2044129,14119478,2770942
46549,28835010,11156755,833 CHESTNUT ST_19107,0.059524,24263264,769 W BLAINE ST STE B_92507,0.857133,5,0.797609,769 W Blaine St Ste B,Department Of Mental Health,Limited To Official Government Duties On,Riverside,CA,92507,24263264,28835010,3596484
250997,38971604,14632224,355 BARD AVE RM 314_10310,0.023770,15686821,83 SAND PIT RD_06810,0.818120,3,0.794350,83 Sand Pit Rd,Childcare Associates,,Danbury,CT,06810,15686821,38971604,4559306
197352,9637845,4023713,7300 N FRESNO ST_93720,0.171384,7214134,6316 N FORKNER AVE_93711,0.962243,4,0.790859,6316 N Forkner Ave,,,Fresno,CA,93711,7214134,9637845,2581510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209091,10048112,15244253,NaN,0.226475,15679734,8101 N HIGH ST_43235,0.732955,5,0.506480,8101 N High St,Ste 130,,Columbus,OH,43235,15679734,10048112,2500572
63232,3035262,15594747,1800 HARRISON ST FL 7_94612,0.313984,879657,1574 WESSEX AVE_94024,0.820446,5,0.506462,1574 Wessex Ave,,,Los Altos,CA,94024,879657,3035262,2090174
160491,7679499,6611424,15 PARKMAN ST_02114,0.151095,13958590,5 E 98TH ST_10029,0.657550,4,0.506455,5 E 98th St,Box 1259,Mount Sinai Hospital,New York,NY,10029,13958590,7679499,2375564
234535,44060636,19554473,5323 HARRY HINES BLVD_75390,0.040909,7072668,1959 NE PACIFIC ST_98195,0.547357,4,0.506448,1959 NE Pacific St,Box 356522,,Seattle,WA,98195,7072668,44060636,5886458


In [149]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20210501.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,14,20,21,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [156]:
BING_SAMPLE = good_bolos[good_bolos.ENTITY_ID.isin(all_things.ENTITY_ID)==False].sort_values('diff', ascending=False).head(6001)

In [157]:
bing_sample = pd.merge(BING_SAMPLE, party_ids, on='PARTY_ID')

In [153]:
ppd['ME'] = fix_me(ppd.ME)

In [160]:
pd.merge(ppd[['MAILING_NAME','ME']],bing_sample, on='ME').to_csv('../../Data/PhoneAppend/BingSample.csv',index=False)

In [162]:
gov = gov.fillna('None')

In [165]:
gov.columns = [x[1:] if x[0]==' ' else x for x in gov.columns.values]

In [167]:
phoned = gov[gov.phn_numbr!='None']

In [169]:
pure = phoned.drop_duplicates('NPI', keep=False)

In [171]:
ppd[['ME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'PRIM_SPEC_CD', 'DESCRIPTION','PE_CD', 'FAX_NUMBER']]

,ME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,PRIM_SPEC_CD,PE_CD,FAX_NUMBER,TOP_CD
0,00102000013,STANFORD,MICHAEL,JAMES,NaN,EM,11,NaN,20
1,00102000021,MARTIN,MERI,KING,NaN,FOP,110,NaN,100
2,00102000030,JACKSON,MARK,THOMAS,NaN,PD,30,8.282522e+09,20
3,00102000048,MAISIAK,NADHIWAN,SAKARIN,NaN,IM,21,NaN,20
4,00102000056,GALA,RAJIV,BABULAL,NaN,OBG,50,5.048430e+09,20
...,...,...,...,...,...,...,...,...,...
1281049,96501960028,SAKULANDA,ROBERT,NaN,NaN,FM,110,NaN,100
1281050,96501960036,MAILA,KAMISHELE,NaN,NaN,IM,22,NaN,20
1281051,96501960044,GAUSI,MWIZA,NaN,NaN,IM,110,NaN,20
1281052,96501960052,MDALA-GAUSI,MASIKU,AGATHA,NaN,US,50,3.367160e+09,20


In [2]:
import pandas as pd
pe_cd = pd.read_excel('../../Data/AMAIA/PRESENT_EMPLOYMENT_CODE_KEY.xlsx')

In [8]:
pe_cd[['present_emp_cd','description']].to_dict()

{'present_emp_cd': {0: 11,
  1: 13,
  2: 21,
  3: 22,
  4: 30,
  5: 35,
  6: 40,
  7: 50,
  8: 63,
  9: 64,
  10: 81,
  11: 82,
  12: 83,
  13: 84,
  14: 85,
  15: 86,
  16: 101,
  17: 110},
 'description': {0: 'Self-Employed Solo Practice                                                ',
  1: 'Two Physician Practice-Full Or Part Owner',
  2: 'Other-Patient Care',
  3: 'Locum Tenens',
  4: 'Group Practice',
  5: 'HMO',
  6: 'Medical School',
  7: 'Non-Government Hospital',
  8: 'City/County/State Government-Hospital',
  9: 'City/County/State Government-Other Than Hospital',
  10: 'Federal Government-Hospital/Army',
  11: 'Federal Government-Hospital/Navy',
  12: 'Federal Government-Hospital/Air Force',
  13: 'Federal Government-Hospital/Usphs',
  14: 'Federal Government-Hospital/Vet Admin',
  15: 'Federal Government-Hospital/Other Agency',
  16: 'Other/Non-Patient Care',
  17: 'No Classification'}}

In [9]:
present_employment_key = {
        11: 'Self-Employed Solo Practice',
        13: 'Two Physician Practice-Full Or Part Owner',
        21: 'Other-Patient Care',
        22: 'Locum Tenens',
        30: 'Group Practice',
        35: 'HMO',
        40: 'Medical School',
        50: 'Non-Government Hospital',
        63: 'City/County/State Government-Hospital',
        64: 'City/County/State Government-Other Than Hospital',
        81: 'Federal Government-Hospital/Army',
        82: 'Federal Government-Hospital/Navy',
        83: 'Federal Government-Hospital/Air Force',
        84: 'Federal Government-Hospital/Usphs',
        85: 'Federal Government-Hospital/Vet Admin',
        86: 'Federal Government-Hospital/Other Agency',
        101: 'Other/Non-Patient Care',
        110: 'No Classification'
    }

In [10]:
listy = [11,13,21,11]
[present_employment_key[x] for x in listy]

['Self-Employed Solo Practice',
 'Two Physician Practice-Full Or Part Owner',
 'Other-Patient Care',
 'Self-Employed Solo Practice']

In [12]:
sample = pd.read_csv('../../Data/POLO_Filter/Filtered_POLOs_Humach_Sample2021-06-15.csv')

In [30]:
list(sample.ME.astype(str))

['102000307',
 '102000749',
 '102001141',
 '102001575',
 '102010612',
 '102021517',
 '102021584',
 '102030249',
 '102030290',
 '102030907',
 '102041275',
 '102050134',
 '102051271',
 '102060768',
 '102061080',
 '102061306',
 '102070330',
 '102070402',
 '102070593',
 '102071239',
 '102071671',
 '102081251',
 '102090501',
 '102091442',
 '102101081',
 '102111010',
 '102111214',
 '102111923',
 '102120311',
 '102121601',
 '102121635',
 '102130359',
 '102130766',
 '102130901',
 '102131266',
 '102131584',
 '102150295',
 '102151046',
 '102151461',
 '102161602',
 '102171705',
 '102700210',
 '102730283',
 '102740548',
 '102741081',
 '102752384',
 '102780710',
 '102780931',
 '102800338',
 '102800842',
 '102801431',
 '102801440',
 '102810589',
 '102810708',
 '102821220',
 '102821653',
 '102831527',
 '102831845',
 '102840925',
 '102850971',
 '102851552',
 '102860071',
 '102861299',
 '102871456',
 '102880161',
 '102880731',
 '102890329',
 '102891490',
 '102900171',
 '102900642',
 '102910982',
 '1029

In [13]:
sample[sample.TELEPHONE_NUMBER!='None']

,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,SUFFIX,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,TELEPHONE_NUMBER,PRIM_SPEC_CD,DESCRIPTION,PE_CD,FAX_NUMBER
0,102000307,MITCHELL,DAVID,BRANTLEY,None,None,P O Box 409703,Atlanta,GA,30384,8509342111.0,EM,Non-Government Hospital,50,8509342114.0
1,102000749,KIM,CHERI,WIGGINS,None,NaN,1395 George Dieter Dr,El Paso,TX,79936,2057505169.0,PM,Group Practice,30,None
2,102001141,BRUCE,EDWARD,TAYLOR,None,NaN,NaN,Montgomery,AL,36108,3348747700.0,IM,Self-Employed Solo Practice,11,3348747700.0
3,102001575,JEFFREY,None,CHOU,None,Ste A,1325 Officers Row,Vancouver,WA,98661,5028953840.0,HO,No Classification,110,None
4,102010612,STEPHANIE,MICHELE,MORGAN,None,NaN,1901 Randolph Rd,Charlotte,NC,28207,2565432273.0,FM,Group Practice,30,2565432293.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,95702990093,IVANA,None,LAZICH,None,Ste 6,6 Essex Dr,Peabody,MA,1960,9785310677.0,NEP,Group Practice,30,9785315676.0
16965,95702990115,TATJANA,None,MARIC,None,NaN,300 68th St SE,Grand Rapids,MI,49548,6164555000.0,CHP,Group Practice,30,6163646400.0
16968,95707010011,TOMISLAV,None,LAKIC,None,NaN,101 S Major St,Eureka,IL,61530,6309156769.0,FM,Non-Government Hospital,50,None
16971,95708850072,ZVJEZDANA,None,DEKIC-DJORDJEVIC,None,Va Cboc,100 Veterans Way,Eglin AFB,FL,32542,5616584836.0,FM,Group Practice,30,4235425109.0


In [176]:
ppd = pd.merge(ppd, pe_cd, left_on='PE_CD', right_on = 'present_emp_cd')

In [14]:
from datetime import date, datetime

In [17]:
previous_release = '210409'

In [24]:
datetime.strptime(previous_release, '%y%m%d')

datetime.datetime(2021, 4, 9, 0, 0)

In [16]:
str(date.today())

'2021-06-15'

In [180]:
pure[['adr_ln_1', 'adr_ln_2', 'cty',
       'st', 'zip', 'phn_numbr']]

,adr_ln_1,adr_ln_2,cty,st,zip,phn_numbr
0,137 LIVINGSTON AVE,None,NEW BRUNSWICK,NJ,089012497,7.32545e+09
1,1670 PUTNAM AVE,None,RIDGEWOOD,NY,113853449,7.18418e+09
2,150 AREA CENTRO MEDICO,None,SAN JUAN,PR,00921,7.87763e+13
5,401 BOGLE ST,SUITE 207,SOMERSET,KY,425032850,6.0668e+09
10,237 AVE LOS VETERANOS,None,LAJAS,PR,006672509,7.87899e+09
...,...,...,...,...,...,...
1324985,BF4 CALLE LA PINTA,None,CAGUAS,PR,007251472,7.87539e+09
1324990,1982 CALLE YUCCA,None,THOUSAND OAKS,CA,913602255,8.18576e+09
1324991,1982 CALLE YUCCA,None,THOUSAND OAKS,CA,913602255,8.18576e+09
1324992,4024 PLANK RD,None,FREDERICKSBURG,VA,224074800,8.66867e+09


In [185]:
purity = pd.merge(pure, party_ids, on='PARTY_ID')

In [187]:
PURITY = pd.merge(purity, ppd, on='ME')

In [240]:
DATA_SAMPLE = POPOP[(POPOP.PARTY_ID.isin(data_bolo_sample.PARTY_ID)==False)&(POPOP.TOP_CD==20)&(POPOP.PHONE!='None')][['adrs_id','NPI','ME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'adr_ln_1', 'adr_ln_2', 'cty',
       'st', 'zip', 'PHONE','PRIM_SPEC_CD', 'description','PE_CD', 'FAX_NUMBER']].sample(10000)

In [242]:
DATA_SAMPLE.to_excel('../../Data/DataGov/National_Downloadable_File_Humach_Sample_25-05-21.xlsx', index=False)

In [199]:
def make_sample(sample):
    purity = pd.merge(sample, party_ids, on='PARTY_ID')
    PURITY = pd.merge(purity, ppd, on='ME')
    
    return(PURITY)

In [237]:
data_bolo = pd.merge(data_bolo_sample, party_ids, on='PARTY_ID')
data_bolo = pd.merge(data_bolo, ppd[['ME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'PRIM_SPEC_CD', 'description','PE_CD', 'FAX_NUMBER','TOP_CD']], on='ME')

In [239]:
data_bolo[(data_bolo.TOP_CD==20)&(data_bolo.ME.isin(DATA_SAMPLE.ME)==False)&(data_bolo.PHONE!='None')].head(5000).to_excel('../../Data/DataGov/National_Downloadable_File_BOLO_Humach_Sample_25-05-21.xlsx', index=False)

In [235]:
data_bolo_sample = TESTER[['PARTY_ID', 'ADDR_1',
       'ADDR_2', 'ADDR_3', 'CITY', 'STATE', 'ZIP', 'COMM_ID', 'ENTITY_ID','PHONE']].head(6000)

In [218]:
TESTER = all_things.drop_duplicates(subset = ['_phn_numbr'])

In [232]:
POPOP = PURITY.drop_duplicates('PHONE')

In [230]:
PURITY['PHONE'] = [str(int(x)) if len(str(int(x)))==10 else 'None'for x in PURITY.phn_numbr]

In [226]:
len('7738993999')

10

In [234]:
TESTER['PHONE'] = [str(int(x)) if len(str(int(x)))==10 else 'None'for x in TESTER['_phn_numbr']]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [243]:
data_bolo[(data_bolo.TOP_CD==20)&(data_bolo.ME.isin(DATA_SAMPLE.ME)==False)&(data_bolo.PHONE!='None')].head(5000).to_excel('../../Data/DataGov/National_Downloadable_File_BOLO_Humach_Sample_Uncleaned_25-05-21.xlsx', index=False)

In [245]:
bingy = pd.read_csv('../../Data/PhoneAppend/BingBing.csv')

In [246]:
bingy['ME']=fix_me(bingy.ME)

In [248]:
bingo = pd.merge(bingy, ppd[['ME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'PRIM_SPEC_CD', 'description','PE_CD', 'FAX_NUMBER','TOP_CD']], on='ME')

In [249]:
bingo.to_excel('../../Data/PhoneAppend/Bing_BOLO_Humach_Sample.xlsx', index=False)

In [1]:
DATA_SAMPLE

NameError: name 'DATA_SAMPLE' is not defined